In [1]:
import csv
import pandas as pd
import numpy as np
import itertools
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz, load_npz
from pandas import read_csv
from scipy.sparse import hstack
from math import exp
from sklearn import preprocessing

In [2]:
sparse_mtx=load_npz('training_sparse_new.npz')

In [3]:
m=sparse_mtx.shape[0]
n=sparse_mtx.shape[1]

In [4]:
data_class=pd.read_csv('solu.csv',header=None)
data_class.shape

(12000, 1)

In [92]:
def weights():
    w=np.zeros((20,61189))
    w[:,:]=0.01 
    w=csr_matrix(w)
    return w



In [16]:
def probX(W,sparse_mtx):
    probarr=W.dot(sparse_mtx.T) 
    probarr=probarr.expm1()
    probarr=norml(probarr)   
    return(probarr)


    

In [17]:
def norml(mtx):
    X_normalized = preprocessing.normalize(mtx, norm='l2',axis=0)
    return X_normalized

In [18]:

def delta_y(y, unique_classes):
    output = np.zeros((len(unique_classes), y.shape[0]),dtype=np.float16)
    for c in unique_classes:
        output[c-1, :] = np.where(y==c, 1., 0.).transpose()                                                                                                                                                                                                                               
    return csr_matrix(output)

In [19]:
def update_weight(w,x,d,learning_rate,penalty):
    #w_new=(w+learning_rate*(d-probX(w,x))*x-penalty*w)
    w_new=(d-probX(w,x))*x
    w_new=w_new-(penalty*w)
    w_new= w + learning_rate*w_new
    
    return(w_new)

In [109]:
def training(X):     
    x = norml(X)
    w=weights() 
    learning_rate=.008
    penalty=.001
    u=np.unique(data_class)
    d=csr_matrix(delta_y(data_class,u))   
    for i in range(15):
        #w_new=update_weight(w,x,d,learning_rate,penalty)
        #print(w_new.shape)
        h = probX(w, x)
        error = (d - h)
        prediction = error * x
        w += learning_rate * (prediction - penalty * w)
       
    return w 

  

In [110]:
# calling training function
weight=training(sparse_mtx)

In [201]:
#testing data 

In [44]:
testdata=pd.read_csv(r'C:\Users\harsh\Desktop\testing.csv',header=None)

In [47]:
documents=testdata.iloc[:,0]

In [48]:
data_new_test=testdata.drop([0],axis=1)

In [49]:
A=np.ones(6774)
data_new_test.insert(0,column=0,value=A)

In [50]:
sparse_mtx_test=csr_matrix(data_new_test)

In [51]:
save_npz('testing_sparse',sparse_mtx_test)

In [52]:
sparse_test=load_npz('testing_sparse.npz')

In [53]:
m=sparse_test.shape[0]
n=sparse_test.shape[1]

In [111]:
#calculate max prob.
def testfunction(Xtest,weight):
    Xtest=norml(Xtest)
    A=probX(weight,Xtest)
    col_argmax = [A.getcol(i).A.argmax() for i in range(A.shape[1])]    
    return col_argmax    

In [112]:
coltest=testfunction(sparse_test,weight)  

In [113]:
#add 1 to class predicted
for i in range(6774):
    coltest[i]=coltest[i]+1

In [115]:
df = pd.DataFrame({"id" : documents, "class" : coltest})
df.to_csv(r'C:\Users\harsh\Desktop\iris.csv', index=False)

In [114]:
coltest

[9,
 7,
 3,
 1,
 4,
 16,
 18,
 1,
 15,
 5,
 10,
 15,
 9,
 14,
 8,
 16,
 1,
 6,
 1,
 15,
 14,
 18,
 9,
 9,
 9,
 15,
 19,
 15,
 2,
 5,
 17,
 6,
 5,
 9,
 4,
 6,
 14,
 3,
 15,
 4,
 7,
 17,
 7,
 14,
 10,
 13,
 17,
 4,
 13,
 16,
 18,
 15,
 19,
 10,
 4,
 4,
 5,
 18,
 17,
 9,
 12,
 2,
 1,
 14,
 11,
 18,
 8,
 14,
 1,
 5,
 12,
 11,
 5,
 19,
 16,
 15,
 15,
 10,
 17,
 5,
 3,
 19,
 4,
 10,
 17,
 20,
 3,
 8,
 8,
 5,
 10,
 20,
 11,
 4,
 16,
 1,
 8,
 4,
 2,
 10,
 13,
 9,
 12,
 16,
 6,
 1,
 12,
 17,
 14,
 16,
 7,
 16,
 11,
 9,
 9,
 14,
 14,
 10,
 17,
 17,
 11,
 2,
 12,
 12,
 14,
 14,
 2,
 11,
 4,
 4,
 6,
 7,
 3,
 15,
 14,
 9,
 6,
 11,
 12,
 18,
 15,
 12,
 8,
 9,
 8,
 7,
 3,
 13,
 18,
 8,
 3,
 14,
 6,
 8,
 16,
 8,
 17,
 15,
 4,
 5,
 10,
 17,
 9,
 19,
 9,
 8,
 18,
 15,
 4,
 4,
 10,
 12,
 16,
 12,
 6,
 9,
 4,
 16,
 10,
 9,
 10,
 9,
 16,
 18,
 16,
 8,
 10,
 18,
 4,
 1,
 5,
 18,
 10,
 19,
 10,
 10,
 12,
 3,
 18,
 18,
 15,
 17,
 7,
 13,
 15,
 14,
 20,
 9,
 19,
 13,
 15,
 8,
 14,
 5,
 18,
 4,
 7,
 11,
 3,
 5,

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20], dtype=int64)

(6774, 61188)

(6774, 61189)